In [4]:
from pystdf.IO import Parser
from pystdf.V4 import pir, ptr, prr
import pandas
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12,8))

In [5]:
help(ptr)

Help on Ptr in module pystdf.V4 object:

class Ptr(pystdf.Types.RecordType)
 |  **Parametric Test Record (PTR)**
 |  
 |  Function:
 |    Contains the results of a single execution of a parametric test in the
 |    test program. The first occurrence of this record also establishes
 |    the default values for all semi-static information about the test,
 |    such as limits, units, and scaling. The PTR is related to the Test
 |    Synopsis Record (TSR) by test number, head number, and site number.
 |  
 |  Data Fields:
 |    ======== ===== ======================================== ====================
 |    Name     Type  Description                              Missing/Invalid Flag
 |    ======== ===== ======================================== ====================
 |    REC_LEN  U*2   Bytes of data following header
 |    REC_TYP  U*1   Record type(15)
 |    REC_SUB  U*1   Record sub-type (10)
 |    TEST_NUM U*4   Test number
 |    HEAD_NUM U*1   Test head number
 |    SITE_NUM U*1   Test

In [9]:
class DataFrameSink(object):
    def __init__(self):
        self.index_tuples = []
        self.series = []
        self.test_nums = []
        self.results = []
        self.limits = {}
        self.data = None
    
    def before_send(self, source, data):
        rectype, fields = data
        #if rectype == pir:
           # self.on_pir(fields)
        if rectype == ptr:
            self.on_ptr(fields)
        #elif rectype == prr:
         #   self.on_prr(fields)
    
    #def on_pir(self, fields):
    #    '''Part Information Record'''
    #    self.test_nums = []
    #    self.results = []

    def on_ptr(self, fields):
        '''Parametric Test Record'''
        self.test_nums.append(fields[ptr.TEST_NUM])
        self.results.append(fields[ptr.RESULT])
        self.limits[fields[ptr.TEST_NUM]] = [fields[ptr.LO_LIMIT], fields[ptr.HI_LIMIT]]
 
    #def on_prr(self, fields):
    #    '''Part Result Record'''
    #    if self.results:
    #        self.series.append(pandas.Series(self.results, self.test_nums))
    #        self.index_tuples.append((int(fields[prr.PART_ID]), fields[prr.SITE_NUM]))
        
    def after_complete(self, source):
        names = ['part', 'site']
        index = pandas.MultiIndex.from_tuples(self.index_tuples, names=names)
        self.data = pandas.DataFrame(self.series, index=index)
        self.data.columns.name = 'test'
        self.limits = pandas.DataFrame(sink.limits, index=['low', 'high']).T
        self.limits.index.name = 'test'
     

In [10]:
f = open('WP6BB19_05_SS_1.stdf', 'rb')
parser = Parser(inp=f)
sink = DataFrameSink()
parser.addSink(sink)
parser.parse()
f.close()
data = sink.data  # data is a pandas DataFrame holding stdf test data
limits = sink.limits  # limits is also a DataFrame, it holds the test limits

In [ ]:
limits[0:10]

In [ ]:
tests = [301260018, 301260019, 301260020, 301260021]
data[tests].hist(bins=20);

In [ ]:
data[tests].describe()